<a href="https://colab.research.google.com/github/NjagiKevin/NjagiKevin/blob/main/Sentiment_Analysis_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification #tokenizer for converting text to sequence of numbers
import torch #argmax to extract our highest seq results
import requests #to grab data from the web
from bs4 import BeautifulSoup #for extracting data
import re #regex function

##Instantiate Model

In [6]:
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

##Encode and Calculate Sentiment

In [7]:
tokens=tokenizer.encode('Ik vond het niet leuk, zou het niet aanraden!', return_tensors='pt')

tokens

tensor([[  101, 23916, 35145, 10184, 11853, 86136, 10167,   117, 16114, 10184,
         11853, 10790, 78161,   106,   102]])

In [8]:
result=model(tokens)

result

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.5439,  3.0094,  1.1987, -2.2135, -3.6231]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [9]:
result.logits #getting the tensor results...

tensor([[ 2.5439,  3.0094,  1.1987, -2.2135, -3.6231]],
       grad_fn=<AddmmBackward0>)

In [10]:
int(torch.argmax(result.logits))+1 #to get the highest value results

2

### Collect reviews

In [11]:
r=requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup=BeautifulSoup(r.text, 'html.parser')
regex=re.compile('.*comment.*')
results=soup.find_all('p', {'class':regex}) #finding all tags in our soup that match our formatting
reviews=[result.text for result in results] #to remove the html tags

In [16]:
reviews

['The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to check on us.. I

## Load Reviews into Dataframe and Score

In [18]:
import pandas as pd
import numpy as np

In [20]:
df= pd.DataFrame(np.array(reviews), columns=['review']) #converting reviews as nupy array

In [21]:
  df.head()

,review
0,The food is fresh and tasty. The scallop cevi...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Visiting from Texas and decided to give this r...


In [22]:
df['review'].iloc[0]

'The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [24]:
def sentiment_score(review):
  tokens=tokenizer.encode(review, return_tensors='pt')
  result=model(tokens) #calculates our sentiment
  return int(torch.argmax(result.logits))+1 #extracts our sentiment scores

In [27]:
sentiment_score(df['review'].iloc[0])

4

In [28]:
df['sentiment']=df['review'].apply(lambda x: sentiment_score(x[:512])) #looping thru our reviews in our reviews column

In [30]:
df['review']

0    The food is fresh and tasty.  The scallop cevi...
1    The food was decent not great..  We had the gu...
2    Food was okay, guacamole was below average. Se...
3    The food and service here was really good.  It...
4    Visiting from Texas and decided to give this r...
5    Don't come here expecting legit Mexican food b...
6    Out of all the restaurants that I tried in Syd...
7    Great atmosphere, attentive service, solid mar...
8    We came here on a Thursday night @ 5pm and by ...
9    Have been here twice and have absolutely loved...
Name: review, dtype: object

In [31]:
df

,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5
5,Don't come here expecting legit Mexican food b...,3
6,Out of all the restaurants that I tried in Syd...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,Have been here twice and have absolutely loved...,5


In [33]:
df['review'].iloc[2]

"Food was okay, guacamole was below average. Service was awful. Waitress acted like she was missing the Finals in The Australian Open. In such a hurry she spilled the fixings for the less than average guac. Took less than a minute to take our order but it wasn't fast enough for her. Felt like we were intruding on her screen time. The food was fair. Prices were way too high for the level of mediocrity in the food. Average food, lousy service Wouldn't recommend this place"